In [23]:
import GPy
import numpy as np
import safeopt
import os
# import hyperspaces as hs
from multiprocessing import Process, Pipe, connection
from typing import List, Tuple, Dict, Final, Callable
from objective_functions import bird_function#, BIRD_FUNCTION_BOUNDS, BIRD_FUNCTION_THRESHOLD


BIRD_FUNCTION_BOUNDS: Final = [
    (-8., 8.),
    (-8., 8.),
]
# BIRD_FUNCTION_BOUNDS: Final = [
#     (-2 * np.pi, 2 * np.pi),
#     (-2 * np.pi, 2 * np.pi)
# ]
BIRD_FUNCTION_THRESHOLD: Final = -35.0

all_parameter_subspaces = []
subspace_indices_for_hyperspace = []
subspaces_deployment_status = []
points_evaluated_in_hyperspace = {}

In [14]:
def create_hyperspaces(
    parameter_spaces: List[Tuple],
    no_subspaces: int
) -> List[List[Tuple]]:
    """Creates hyperspaces for the given parameters by dividing each of them into given number of subspaces.

    Args:
        parameter_spaces:
            List of tuple containing two items i.e: lower bound and upper bound for each search parameter.
        no_subspaces:
            How many number of subspaces to create for a search parameter.

    Returns:
        hyperspaces:
            Set of all possible combinations of subspaces.
    """
    global subspaces_deployment_status
    global all_parameter_subspaces
    global subspace_indices_for_hyperspace
    # to divide each parameter space into given number of subspaces
    for parameter_space in parameter_spaces:
        low, high = parameter_space
        subspace_length = abs(high - low)/no_subspaces
        parameter_subspaces = []
        for i in range(no_subspaces):
            parameter_subspaces.append((low, low + subspace_length))
            low = low + subspace_length
        all_parameter_subspaces.append(parameter_subspaces)

    rows = len(all_parameter_subspaces)  # no_parameters
    columns = len(all_parameter_subspaces[0])  # no_subspaces

    # initializing deployed status for each subsapce
    for i in range(rows):
        each_parameter_subspaces = []
        for _ in range(columns):
            # for each subspace maintain these 3 states to determine its deployment status
            # [is this subspace deployed (True/False), associated with any subspace (True/False),
            #  hyperspace number (None-not deployed with any hyperspace/int-deployed with that hyperspace)]
            each_parameter_subspaces.append(False)
        subspaces_deployment_status.append(each_parameter_subspaces)

    # no_hyperspaces = no_subspaces ** no_parameters
    hyperspaces = []  # contains all possible combinations of subspaces
    for _ in range(columns**rows):
        hyperspaces.append([])
        subspace_indices_for_hyperspace.append([])

    for row in range(rows):
        repeat = columns ** (rows-row-1)
        for column in range(columns):
            item = all_parameter_subspaces[row][column]
            start = column * repeat
            for times in range(columns**row):
                for l in range(repeat):
                    hyperspaces[start+l].append(item)
                    subspace_indices_for_hyperspace[start+l].append(column)
                start += columns * repeat
    return hyperspaces


In [3]:
def which_hyperspace(
    x: List[List],
    hyperspaces: List[List[Tuple]]
) -> Dict:
    """Creates a dictionary with hyperspace number as key and list of points belong to that hyperspace as 
    corresponding value.

    Args:
        x:
            List of points.
        hyperspaces:
            List containing all hyperspaces.

    Returns:
        safe_hyperspaces:
            Dictionary with hyperspace number and list of points as key-value pair.
    """
    safe_hyperspaces = {}

    for point in x:
        for i, hyperspace in enumerate(hyperspaces):
            belongs_flag = True
            for dimension, value in enumerate(point):
                if value >= hyperspace[dimension][0] and value < round(hyperspace[dimension][1], 4):
                    continue
                belongs_flag = False
            if belongs_flag:
                if i not in safe_hyperspaces:
                    safe_hyperspaces[i] = [point]
                else:
                    safe_hyperspaces[i].append(point)
    return safe_hyperspaces

In [4]:
def optimization(
    x: List[List],
    y: List[List], 
    bounds: List[Tuple],
    kernel: GPy.kern,
    objective_function: Callable,
    safe_threshold: float,
    noise_var: float,
    safe_hyperspace_no: int,
    hyperspaces: List[List[Tuple]],
    conn: connection.Connection
) -> None:
    """
    """
    # current_hyperspace = list(safe_hyperspaces.keys())[0]
    current_hyperspace = safe_hyperspace_no
#     print("current_hyperspace", current_hyperspace)

    x = np.array(x)
    # The statistical model of our objective function
    if y is None:
        gp = GPy.models.GPRegression(
            x, objective_function(x),
            kernel, noise_var=noise_var
        )
    else:
        y = np.array(y)
        gp = GPy.models.GPRegression(
            x, y,
            kernel, noise_var=noise_var
        )

    # The optimization routine
    opt = safeopt.SafeOptSwarm(
        gp,
        BIRD_FUNCTION_THRESHOLD,
        bounds=bounds,
        threshold=0.2
    )
    # opt = safeopt.SafeOpt(
    #     gp,
    #     parameter_set,
    #     safe_threshold,
    #     lipschitz=None,
    #     threshold=0.2
    # )

    # {key: hyperspace_no, value: evaluated unsfae points}
    # converting to `np.arry()` or `list` because `opt.x` is of `ObsAr` type. 
    if y is None:
        evaluated_points = {current_hyperspace: [[np.array(x_i) for x_i in opt.x], list(opt.y)]}
    else:
        evaluated_points = {}

    try:
        for i in range(5):
            # obtain new query point
#             print("Iteration:", i)
            x_next = opt.optimize()
#             print("x_next:", x_next)

            # Get a measurement from the real system
            y_meas = objective_function(x_next)
#             print("y_meas: ", y_meas)

            new_hyperspace = list(which_hyperspace([x_next], hyperspaces))[0]
#             print("new hyperspace", new_hyperspace)

            # CHECK : whether to compare `y_meas` with `threshold` to confirm for safety and add to corresponding 
            # hyperspace. Since non-safe point also provides some information about objective function.
            if new_hyperspace in evaluated_points.keys():
                evaluated_points[new_hyperspace][0].append(x_next)
                evaluated_points[new_hyperspace][1].append(y_meas[0])
            else:
                evaluated_points[new_hyperspace] = [[x_next], [y_meas[0]]]
                
            if y_meas >= safe_threshold and new_hyperspace != current_hyperspace:
                # conn.send((current_hyperspace, new_hyperspace, evaluated_points))
                # conn.close()
                break

            # Add this to the GP model
            opt.add_new_data_point(x_next, y_meas)

            # opt.plot(100, plot_3d=False)
    finally:
        conn.send((current_hyperspace, new_hyperspace, evaluated_points))
        conn.close()

In [5]:
def split_search_space(ss, hs1, hs2):
#     print()
#     print(ss)
#     print(hs1, hs2)
    ss1 = []
    ss2 = []
    for param_index, (i, j) in enumerate(zip(hs1, hs2)):
#         print(param_index)
#         print(i, j)
#         print(ss[param_index])
        start, end = ss[param_index]
        if i == j:
            ss1.append(ss[param_index])
            ss2.append(ss[param_index])
        else:
            if i > j:
                i, j = j, i
            ds = subspaces_deployment_status[param_index]
            ds[i] = True
            ds[j] = True
            in_between = j - i - 1
            hs1_right = in_between // 2
            hs2_left = in_between - hs1_right
            hs1_low = i
            hs1_high = i + hs1_right
            hs2_low = j - hs2_left
            hs2_high = j

            for k in range(i - 1, start - 1, -1):
                if not ds[k]:
                    hs1_low = k
                else:
                    break

            for k in range(j + 1, end + 1):
                if not ds[k]:
                    hs2_high = k
                else:
                    break

            ss1.append([hs1_low, hs1_high])
            ss2.append([hs2_low, hs2_high])
            break
    if param_index < len(ss) - 1:
        for i in range(param_index + 1, len(ss)):
            ss1.append(ss[i])
            ss2.append(ss[i])
    return ss1, ss2

In [6]:
def get_bounds_from_index(
    hyperspace_bounds: List[Tuple]
) -> List[Tuple]:
    """
    """
    bounds = []
    for parameter_no, bound in enumerate(hyperspace_bounds):
        parameter_subspaces = all_parameter_subspaces[parameter_no]
        low = parameter_subspaces[bound[0]][0]
        high = parameter_subspaces[bound[1]][1]
        bounds.append((low, high))
    return bounds

In [7]:
def deploy_hyperspace(
    hyperspace_no: int, 
    hyperspace_bounds: List[Tuple],
    kernel: GPy.kern,
    objective_function: Callable,
    safe_threshold: float,
    noise_var: float,
    hyperspaces: List[List[Tuple]]
) -> None:
    """
    """
    print(os.getpid(), "deploy new hyperspace", hyperspace_no)
    if hyperspace_no not in points_evaluated_in_hyperspace.keys():
        print("No safe points given for hyperspace:", hyperspace_no)
        return
    x, y = points_evaluated_in_hyperspace[hyperspace_no]
#     print(x, y)
    bounds = get_bounds_from_index(hyperspace_bounds)
#     print("bounds : ", bounds)
    parent_conn, child_conn = Pipe()
    p = Process(target=optimization, args=(x, y, bounds, kernel, objective_function, safe_threshold, 
    noise_var, hyperspace_no, hyperspaces, child_conn))

    p.start()
    current_hyperspace, new_hyperspace, evaluated_points = parent_conn.recv()
    print(current_hyperspace, new_hyperspace, evaluated_points)
    for k, v in evaluated_points.items():
        if k in points_evaluated_in_hyperspace.keys():
            for i, value in enumerate(v):
                for point in value:
                    points_evaluated_in_hyperspace[k][i].append(point)
        else:
            points_evaluated_in_hyperspace[k] = v
    hs1 = subspace_indices_for_hyperspace[current_hyperspace]
    hs2 = subspace_indices_for_hyperspace[new_hyperspace]
    ss1, ss2 = split_search_space(hyperspace_bounds, hs1, hs2)
    deploy_hyperspace(current_hyperspace, ss1, kernel, objective_function, safe_threshold, noise_var, hyperspaces)
    deploy_hyperspace(new_hyperspace, ss2, kernel, objective_function, safe_threshold, noise_var, hyperspaces)

In [8]:
def deploy_whole_space(
    x: List[List],
    bounds: List[Tuple], 
    bounds_indices: List[Tuple], 
    kernel: GPy.kern,
    objective_function: Callable,
    safe_threshold: float,
    noise_var: float,
    hyperspaces: List[List[Tuple]]
) -> None:
    """
    """
    safe_hyperspace_no = list(which_hyperspace(x, hyperspaces))[0]
    parent_conn, child_conn = Pipe()
    p = Process(target=optimization, args=(x, None, bounds, kernel, objective_function, safe_threshold, 
    noise_var, safe_hyperspace_no, hyperspaces, child_conn))

    p.start()
    current_hyperspace, new_hyperspace, evaluated_points = parent_conn.recv()
    print(current_hyperspace, new_hyperspace, evaluated_points)
    for k, v in evaluated_points.items():
        points_evaluated_in_hyperspace[k] = v
    print(os.getpid(), points_evaluated_in_hyperspace)
    p.join()
    # call split search space method
    hs1 = subspace_indices_for_hyperspace[current_hyperspace]
    hs2 = subspace_indices_for_hyperspace[new_hyperspace]
    ss1, ss2 = split_search_space(bounds_indices, hs1, hs2)
    deploy_hyperspace(current_hyperspace, ss1, kernel, objective_function, safe_threshold, noise_var, hyperspaces)
    deploy_hyperspace(new_hyperspace, ss2, kernel, objective_function, safe_threshold, noise_var, hyperspaces)

In [9]:
def initial_deploy(
    x: List[List],
    bounds: List[Tuple],
    bounds_indices: List[Tuple], 
    kernel: GPy.kern,
    objective_function: Callable,
    safe_threshold: float,
    noise_var: float,
    hyperspaces: List[List[Tuple]]
) -> None:
    """
    """
    if x.shape[0] == 1:
        # if safe set contains only one point, then we have to deploy whole space to a process.
        deploy_whole_space(x, bounds, bounds_indices, kernel, objective_function, safe_threshold, 
        noise_var, hyperspaces)
    elif x.shape[0] != 0:
        # if safe set is not empty, deploy corresponding hyperspaces for given points in safe set.
        safe_hyperspaces = which_hyperspace(x, hyperspaces)
        # deploy_hyperspace(safe_hyperspaces, hyperspaces)


In [10]:
bounds = BIRD_FUNCTION_BOUNDS
no_subspaces = 4
bounds_indices = [(0, no_subspaces-1) for _ in range(len(bounds))]
safe_threshold = BIRD_FUNCTION_THRESHOLD
# parameter_set = safeopt.linearly_spaced_combinations(bounds, 1000)
hyperspaces = create_hyperspaces(bounds, no_subspaces)

# Measurement noise
noise_var = 0.05 ** 2

# Initial safe set
x0 = np.zeros((1, len(bounds)))  # safe point at zero
# x0 = np.array([[-2.5]]) # 1D single safe point
# x0 = np.array([[-2.5, 3.4]]) # 2D single safe point
# x0 = np.array([[-2.5], [6.5]]) # 1D multiple safe points

# Define Kernel
kernel = GPy.kern.RBF(
    input_dim=len(bounds),
    variance=2.,
    lengthscale=1.0,
    ARD=True
)

# true function
objective_function = bird_function

In [11]:
initial_deploy(x0, bounds, bounds_indices, kernel, objective_function, safe_threshold, noise_var, hyperspaces)

10 11 {10: [[array([0., 0.])], [array([-2.71828183])]], 11: [[array([1.45291555, 6.28318531])], [array([-23.65119756])]]}
99795 {10: [[array([0., 0.])], [array([-2.71828183])]], 11: [[array([1.45291555, 6.28318531])], [array([-23.65119756])]]}
99795 deploy new hyperspace 10
10 14 {14: [[array([5.90778906, 3.14159265])], [array([-10.18082978])]]}
99795 deploy new hyperspace 10
10 0 {10: [[array([-6.28318531, -6.28318531]), array([-6.28318531, -6.10208357]), array([-6.28318531, -6.28318531])], [array([2.71828183]), array([-4.32176685]), array([-54.63974981])]], 0: [[array([-6.28318531, -6.28318531])], [array([-2.71828183])]]}
99795 deploy new hyperspace 10
10 0 {0: [[array([-6.28318531, -4.89528733])], [array([-3.90985971])]]}
99795 deploy new hyperspace 10


Process Process-5:
Traceback (most recent call last):
  File "/tmp/ipykernel_99795/2749451234.py", line 59, in optimization
    x_next = opt.optimize()
  File "/home/shiva/Documents/bo-prog/distributed-safe-bo/dbo-env/lib/python3.8/site-packages/safeopt/gp_opt.py", line 1153, in optimize
    x_maxi, std_maxi = self.get_new_query_point('maximizers')
  File "/home/shiva/Documents/bo-prog/distributed-safe-bo/dbo-env/lib/python3.8/site-packages/safeopt/gp_opt.py", line 1086, in get_new_query_point
    swarm.run_swarm(self.max_iters)
  File "/home/shiva/Documents/bo-prog/distributed-safe-bo/dbo-env/lib/python3.8/site-packages/safeopt/swarm.py", line 134, in run_swarm
    values, safe = self.fitness(self.positions)
  File "/home/shiva/Documents/bo-prog/distributed-safe-bo/dbo-env/lib/python3.8/site-packages/safeopt/gp_opt.py", line 929, in _compute_particle_fitness
    mean, var = self.gps[0].predict_noiseless(particles)
  File "/home/shiva/Documents/bo-prog/distributed-safe-bo/dbo-env/lib/p

KeyboardInterrupt: 

In [ ]:
bounds = [(-8, 8), (-8, 8)]

[[array([0., 0.]),
  array([ 0.        , -3.12863572]),
  array([0., 0.]),
  array([0., 0.]),
  array([0., 0.]),
  array([0., 0.]),
  array([0., 0.])],
 [array([-2.71828183]),
  array([-12.56095375]),
  array([-2.71828183]),
  array([-2.71828183]),
  array([-2.71828183]),
  array([-2.71828183]),
  array([-2.71828183])]]